<a href="https://colab.research.google.com/github/suzitmm/mid_project/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from torchvision.io import read_image
import json

In [2]:
!git clone https://github.com/CHILab1/MedPix-2.0.git

Cloning into 'MedPix-2.0'...
remote: Enumerating objects: 2169, done.
remote: Counting objects: 100% (2169/2169), done.
remote: Compressing objects: 100% (2168/2168), done.
remote: Total 2169 (delta 70), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (2169/2169), 308.16 MiB | 26.06 MiB/s, done.
Resolving deltas: 100% (70/70), done.
Updating files: 100% (2065/2065), done.


1) Dataset

In [47]:
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class MedPixDataset(Dataset):
    def __init__(self, path, stage):
      if stage == 'train':
        self.path_image = path + 'images/'
        self.path_label = path + 'splitted_dataset/descriptions_train.jsonl'

      if stage == 'test':
        self.path_image = path + 'images/'
        self.path_label = path + 'splitted_dataset/descriptions_test.jsonl'

      with open(self.path_label, 'r') as f:
        self.data_label = [json.loads(line) for line in f] #load the dictonary and assigns to data_label

    def __len__(self):
      return len(self.data_label)

    def __getitem__(self, idx):

      label = self.data_label[idx]

      image = Image.open(self.path_image + label['image'] + '.png').convert('RGB')
      transform = transforms.Compose([
        transforms.Resize((256, 256)),  #Resize to 256x256
        transforms.ToTensor()
      ]) #transform in tensors so it can be stacked in a batch
      image = transform(image)
      category = label['Location Category'] #Access 'Location Category' from the dictionary stored i data_label

      return  category, image

In [44]:
image_shapes = [image.shape for category, image in train_dataset]
image_shapes[:3]

[torch.Size([3, 256, 256]),
 torch.Size([3, 256, 256]),
 torch.Size([3, 256, 256])]

In [46]:
from collections import Counter
Counter(image_shapes)

Counter({torch.Size([3, 256, 256]): 1394, torch.Size([4, 256, 256]): 259})

In [48]:
path = '/content/MedPix-2.0/'

train_dataset = MedPixDataset(path, 'train')

test_dataset = MedPixDataset(path, 'test')

example1 = train_dataset[7]

example1

('Head',
 tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0000, 0.0000],
          ...,
          [0.0039, 0.0078, 0.0078,  ..., 0.0039, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039]],
 
         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0000, 0.0000],
          ...,
          [0.0039, 0.0078, 0.0078,  ..., 0.0039, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039]],
 
         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.00

2) Dataloader

The Dataset retrieves our dataset’s features and labels one sample at a time. While training a model, we typically want to pass samples in “minibatches”, reshuffle the data at every epoch to reduce model overfitting, and use Python’s multiprocessing to speed up data retrieval.

In [49]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=3, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=3, shuffle=True)

for batch_size in train_dataloader:
    #print(batch_size)
    print('Images:', batch_size[1])
    print('Labels:', batch_size[0])
    break

Images: tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.7647, 0.7725, 0.7725],
          [0.0000, 0.0000, 0.0000,  ..., 0.7490, 0.7725, 0.7843],
          [0.0000, 0.0000, 0.0000,  ..., 0.7216, 0.7608, 0.7569],
          ...,
          [0.0039, 0.0039, 0.0039,  ..., 0.0235, 0.0275, 0.0275],
          [0.0039, 0.0039, 0.0039,  ..., 0.0078, 0.0078, 0.0078],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.7647, 0.7725, 0.7725],
          [0.0000, 0.0000, 0.0000,  ..., 0.7490, 0.7725, 0.7843],
          [0.0000, 0.0000, 0.0000,  ..., 0.7216, 0.7608, 0.7569],
          ...,
          [0.0039, 0.0039, 0.0039,  ..., 0.0235, 0.0275, 0.0275],
          [0.0039, 0.0039, 0.0039,  ..., 0.0078, 0.0078, 0.0078],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.7647, 0.7725, 0.7725],
          [0.0000, 0.0000, 0.0000,  ..., 0.7490, 0.7725, 0.7843],
          [0.0000, 0.0000, 0.0000,